# 시카고 데이터 생성

- 테이블 생성

In [20]:
import mysql.connector as mc

remote = mc.connect(
    host = 'localhost',
    user = 'robot',
    password = '1234',
    database = 'armbase',
)

cur = remote.cursor(buffered=True)
sql = '''CREATE TABLE sandwich (
ranking INT,
cafe VARCHAR(32),
menu VARCHAR(64),
price FLOAT,
address VARCHAR(34)
)'''

cur.execute(sql)
remote.commit()

- 데이터 url 불러오기

In [21]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urljoin # 절대주소와 상대주소를 구분하기 위해 사용
## user agent 활용
from user_agent import generate_user_agent

def open_html(url):
    req = Request(url=url, headers={'User-Agent':generate_user_agent()})
    res = urlopen(req)
    return BeautifulSoup(res, 'html.parser')

url = 'https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/'
soup = open_html(url)

# print(soup.prettify())

- 데이터 입력

In [22]:
from tqdm.notebook import tqdm

import mysql.connector as mc

remote = mc.connect(
    host = 'localhost',
    user = 'robot',
    password = '1234',
    database = 'armbase',
)

cur = remote.cursor(buffered=True)
sql = '''INSERT INTO sandwich VALUES (%s, %s, %s, %s, %s)'''

prefix = 'http://www.chicagomag.com'

pbar = tqdm(soup.find_all('div', 'sammy'),
            total=len(soup.find_all('div', 'sammy')),
            # ncols=80,
            leave=True)

for each in pbar:
    rank, menu, name, _ = each.get_text().strip().split('\n')
    href = urljoin(prefix, each.find('a')['href'])
    store_soup = open_html(href)
    text = store_soup.find('p', 'addy').get_text().strip().split(',')[0]
    price = re.search('\$\d+.(\d+)?', text).group()
    address = text[len(price)+1:]
    cur.execute(sql, (int(rank), name, menu, float(price[1:]), address))

    pbar.set_description(f'{name} Searching')

remote.commit()
remote.close()

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
remote = mc.connect(
    host = 'localhost',
    user = 'robot',
    password = '1234',
    database = 'armbase',
)

cur = remote.cursor()
cur.execute('select * from sandwich')
result = cur.fetchall()
remote.close()
df = pd.DataFrame(result, columns=['ranking', 'cafe', 'menu', 'price', 'address'])
df.head()


,ranking,cafe,menu,price,address
0,1,Old Oak Tap,BLT,10.0,2109 W. Chicago Ave.
1,2,Au Cheval,Fried Bologna,9.0,800 W. Randolph St.
2,3,Xoco,Woodland Mushroom,9.5,445 N. Clark St.
3,4,Al’s Deli,Roast Beef,9.4,914 Noyes St.
4,5,Publican Quality Meats,PB&L,10.0,825 W. Fulton Mkt.
